In [1]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

from library_dicom.dicom_processor.tools.folders import *

from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *

import numpy as np
import SimpleITK as sitk

import csv

In [2]:
csv_file = '/home/salim/Documents/DeepOncopole/MedicalSegmentation_10082020/vol_lymphoma_mask_before_after_resample.csv'

with open(csv_file, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] #enlever première ligne

In [3]:
print(csv_data[0])

['0', '/media/salim/DD 2To/AHL2011_NIFTI/1.2.124.113532.10.149.172.5.20131107.93455.7725981_nifti_PT.nii', '/media/salim/DD 2To/AHL2011_NIFTI/1.2.124.113532.10.149.172.5.20131107.93455.7725981_nifti_CT.nii', '/media/salim/DD 2To/AHL2011_NIFTI/1.2.124.113532.10.149.172.5.20131107.93455.7725981_nifti_mask.nii', '539128.3736253536', '536592.384']


In [4]:
volume = []
for item in csv_data : 
    volume.append(float(item[4]))
print(volume[0])
for i in range(len(volume)) :
    volume[i] = volume[i] * 10**(-3)

print(volume[0])

539128.3736253536
539.1283736253536


In [5]:
study_uid = []
for item in csv_data : 
    study_uid.append(item[1].split('/')[5].split('_')[0])

In [41]:
print(study_uid[0])

1.2.124.113532.10.149.172.5.20131107.93455.7725981


In [7]:
csv_file_2 = '/media/salim/DD 2To/AHL2011_CSV_DATA/pet0.csv'

with open(csv_file_2, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ';') #liste pour chaque ligne 
    csv_data_2 = []
    for row in reader :
        csv_data_2.append(row)
        
del csv_data_2[0] #enlever première ligne

In [43]:
csv_data_2[1]

['CON,DA',
 '13011101331015',
 '1.2.124.113532.10.149.172.5.20131107.93455.7725981',
 '4',
 'con_da_nov 7_2013.csv',
 '5',
 '13.79',
 'Mediastinal',
 '15.5',
 'Mediastinal',
 '1.4',
 '1.6',
 '2.08',
 '2.17',
 '3',
 '1',
 '5',
 '2.62',
 '4',
 '2.5',
 '2']

In [8]:
csv_directory = '/media/salim/DD 2To/AHL2011_CSV_DATA/AHL2011-CSV'
list_csv = []
for uid in study_uid : 
    for data in csv_data_2 : 
        if uid in data : 
            list_csv.append(os.path.join(csv_directory, data[4]))
            break


In [9]:
print(len(list_csv))

355


In [17]:
list_csv[0]

'/media/salim/DD 2To/AHL2011_CSV_DATA/AHL2011-CSV/con_da_nov 7_2013.csv'

In [35]:
def csv_reader_volume(csv_path) :
    
    with open(csv_path, 'r') as csv_file :
        reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
        csv_data = []
        for row in reader :
            csv_data.append(row)

    for i in range(len(csv_data)) :
        #print(i)
        #print(csv_data[i])
        if ' sum' in csv_data[i] :
            return float(csv_data[i][3])


In [36]:
csv_reader_volume(list_csv[0])

564.63

In [37]:
volume_csv_original = []
for csv_file in list_csv : 
    volume_csv_original.append(csv_reader_volume(csv_file))


In [38]:
print(volume_csv_original)

[564.63, 130.75, 92.8, 133.44, 163.71, 434.24, 230.91, 427.39, 627.58, 187.97, 122.05, 113.11, 370.74, 167.85, 83.22, 353.99, 159.71, 52.05, 323.49, 628.39, 78.85, 59.28, 32.04, 243.44, 128.38, 106.92, 81.54, 57.8, 48.45, 42.53, 149.79, 122.29, 88.87, 110.25, 225.73, 1109.65, 89.06, 234.76, 49.33, 216.61, 231.72, 1302.68, 106.07, 1861.35, 183.72, 140.16, 137.41, 587.58, 598.34, 216.96, 606.85, 159.42, 271.17, 464.06, 247.74, 51.38, 280.29, 140.31, 77.1, 310.11, 267.52, 528.19, 572.8, 56.45, 145.06, 416.42, 234.98, 59.66, 94.67, 390.6, 100.37, 363.22, 129.43, 92.32, 122.76, 202.85, 72.66, 224.96, 155.01, 86.27, 266.94, 187.65, 136.96, 46.72, 386.88, 82.24, 401.65, 932.61, 128.7, 297.98, 37.12, 531.07, 217.02, 257.01, 352.96, 58.37, 261.44, 654.34, 159.62, 352.9, 610.18, 41.92, 179.16, 223.17, 1134.73, 429.91, 89.23, 277.43, 459.25, 121.55, 114.75, 294.92, 193.15, 33.98, 401.84, 247.52, 119.02, 576.9, 136.82, 49.34, 270.08, 1173.42, 53.76, 268.45, 206.12, 162.13, 200.83, 782.91, 129.6, 2

In [40]:
nifti_directory = '/media/salim/DD 2To'

with open(os.path.join(nifti_directory, 'AHL2011_volume.csv'), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["CSV_VOLUME", "CALCULATED_VOLUME_BEFORE_RESAMPLE"])
    for i in range(len(volume_csv_original)) :
        csv_writer.writerow([volume_csv_original[i], volume[i]])